In [4]:
import os
import nibabel as nib
import numpy as np
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
# sam mask转换为nnunet的格式，避免数据分布问题
'''
对齐ground truth和sam出来的mask
GT label脊柱向右，和原始图像一致。
SAMlabel脊柱在上，并且左右翻转，和原始图像不一致。
因此602中训练的mask需要向右转90°并左右翻转，和原始mask一致。
以期预测时mask和GT mask一致。
'''
def flip_slices_90_single_file(p):
    filepath=p
    img = nib.load(filepath)
    data = img.get_fdata()
    print(data.shape)
    flipped_data = np.rot90(data, 1, axes=(0, 1))
            # Flip each slice left-right
    flipped_data = np.fliplr(flipped_data)
    flipped_img = nib.Nifti1Image(flipped_data, img.affine, img.header)   
    # Save the new image to a file
    # flipped_filepath = os.path.join(folder, 'flipped_' + filename)
    nib.save(flipped_img, filepath)
    print('Saved flipped image to', filepath)

    
def flip_slices_90(folder):
    for filename in os.listdir(folder):
        if filename.endswith('.nii.gz'):
            filepath = os.path.join(folder, filename)
            img = nib.load(filepath)
            data = img.get_fdata()
            print(data.shape)
            # transposed_data = np.transpose(data, (2, 0, 1))
            # Flip each slice 180 degrees

            # 将输入图像归一化并且右转90°
            # flipped_data = np.rot90(data, 1, axes=(1, 2))

            # axes可以参考3D_dataset.ipynb中的探索部分, channel在第2维，因此旋转第0维和第1维
            flipped_data = np.rot90(data, 1, axes=(0, 1))
            # Flip each slice left-right
            flipped_data = np.fliplr(flipped_data)
            
            # Normalize intensity to 0-255
            # normalized_data = ((flipped_data - flipped_data.min()) * (255 - 0)) / (flipped_data.max() - flipped_data.min()) + 0
            
            # Move the channel axis from 0 to 2
            # flipped_data = np.transpose(normalized_data, (1, 2, 0))
            # print(flipped_data.shape)
            
            # Create a new NIfTI image with the flipped data
            flipped_img = nib.Nifti1Image(flipped_data, img.affine, img.header)
            
            # Save the new image to a file
            # flipped_filepath = os.path.join(folder, 'flipped_' + filename)
            nib.save(flipped_img, filepath)
            print('Saved flipped image to', filepath)

# Replace 'your_folder' with the path to your folder
# flip_slices_90(join(nnUNet_raw, 'Dataset602_LUNA/labelsTr/'))
flip_slices_90_single_file(join(nnUNet_raw, 'Dataset602_LUNA/labelsTr/IMG_0432.nii.gz'))

# flip_slices_180(join(nnUNet_raw, 'Dataset501_LUNA/imagesTs'))
# flip_slices_180(join(nnUNet_raw, 'Dataset601_LUNA/imagesTs_predfulres'))


(512, 512, 513)
Saved flipped image to /home/xulei/projects/wei/2DSAM3D/nnUNet/nnUNetFrame/DATASET/nnUNet_raw/Dataset602_LUNA/labelsTr/IMG_0432.nii.gz


In [ ]:
import os
import nibabel as nib
import numpy as np
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
# 将手动标注的数据(4 cases)转换为nnunet的格式（channel的位置和上下颠倒问题），避免模型因为数据分布问题预测不好
'''
对于形态学方法出来的mask，需要将其转换为nnunet的格式，即channel的位置和上下颠倒问题
原始图像脊柱在右边，强度没有归一化（训练时归一化了），预测出来的label是向右转90°并左右翻转的（即脊柱在下）。
测试图片强度没有归一化，预测出来的label是完全一样的，没有旋转。
因此600测试时，需先将输入图像归一化并且右转90°，脊柱在右，以期分布一致。
预测出来的mask向左转180°并左右翻转（向左90左右翻：回归右转90输入图像的mask，再向左90回归原始输入图像，即GT对应图像），以期和GT正常比较。

'''
def flip_slices_180(folder):
    for filename in os.listdir(folder):
        if filename.endswith('.nii.gz'):
            filepath = os.path.join(folder, filename)
            img = nib.load(filepath)
            data = img.get_fdata()
            print(data.shape)
            # transposed_data = np.transpose(data, (2, 0, 1))
            # Flip each slice 180 degrees

            # 将输入图像归一化并且右转90°
            # flipped_data = np.rot90(data, 1, axes=(1, 2))
            flipped_data = np.rot90(data, 2, axes=(0, 1))
            # Flip each slice left-right
            # flipped_data = np.fliplr(data)
            
            # Normalize intensity to 0-255
            # normalized_data = ((flipped_data - flipped_data.min()) * (255 - 0)) / (flipped_data.max() - flipped_data.min()) + 0
            
            # Move the channel axis from 0 to 2
            # flipped_data = np.transpose(normalized_data, (1, 2, 0))
            # print(flipped_data.shape)
            
            # Create a new NIfTI image with the flipped data
            flipped_img = nib.Nifti1Image(flipped_data, img.affine, img.header)
            
            # Save the new image to a file
            # flipped_filepath = os.path.join(folder, 'flipped_' + filename)
            nib.save(flipped_img, filepath)
            print('Saved flipped image to', filepath)

# Replace 'your_folder' with the path to your folder
flip_slices_180(join(nnUNet_raw, 'Dataset601_LUNA/imagesTs'))
flip_slices_180(join(nnUNet_raw, 'Dataset501_LUNA/imagesTs'))
# flip_slices_180(join(nnUNet_raw, 'Dataset601_LUNA/imagesTs_predfulres'))


In [ ]:
import os
import nibabel as nib
import numpy as np
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

In [2]:

# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    # perform_everything_on_gpu=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
)
# initializes the network architecture, loads the checkpoint
predictor.initialize_from_trained_model_folder(
    join(nnUNet_results, 'Dataset601_LUNA/nnUNetTrainer__nnUNetPlans__3d_fullres'),
    use_folds=('1'),
    checkpoint_name='checkpoint_final.pth',
)
# variant 1: give input and output folders
predictor.predict_from_files(join(nnUNet_raw, 'Dataset601_LUNA/Ts'),
                                join(nnUNet_raw, 'Dataset601_LUNA/Ts_predfulres'),
                                save_probabilities=False, overwrite=False,
                                num_processes_preprocessing=2, num_processes_segmentation_export=2,
                                folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 32 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 32 cases that I would like to predict
overwrite was set to False, so I am only working on cases that haven't been predicted yet. That's 32 cases.

Predicting IMG_0501:
perform_everything_on_device: True


 55%|█████▌    | 55/100 [00:21<00:17,  2.53it/s]


KeyboardInterrupt: 

In [ ]:
# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    # perform_everything_on_gpu=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
)
# initializes the network architecture, loads the checkpoint
predictor.initialize_from_trained_model_folder(
    join(nnUNet_results, 'Dataset501_LUNA/nnUNetTrainer__nnUNetPlans__3d_fullres'),
    use_folds=('1'),
    checkpoint_name='checkpoint_final.pth',
)
# variant 1: give input and output folders
predictor.predict_from_files(join(nnUNet_raw, 'Dataset501_LUNA/Ts'),
                                join(nnUNet_raw, 'Dataset501_LUNA/Ts_predfulres'),
                                save_probabilities=False, overwrite=False,
                                num_processes_preprocessing=2, num_processes_segmentation_export=2,
                                folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

In [ ]:
# 预测出来的mask向左转180°并左右翻转（向左90左右翻：回归右转90输入图像的mask，再向左90回归原始输入图像，即GT对应图像），以期和GT正常比较。
# 
def left_turn_180_and_fliplr(folder):
    for filename in os.listdir(folder):
        if filename.endswith('.nii.gz'):
            filepath = os.path.join(folder, filename)
            img = nib.load(filepath)
            data = img.get_fdata()
            print(data.shape)
            # transposed_data = np.transpose(data, (2, 0, 1))
            # Flip each slice 180 degrees

            #mask左转180°
            # flipped_data = np.rot90(data, 2, axes=(0, 1))
            # Flip each slice left-right
            # flipped_data = np.fliplr(flipped_data)
            
            # Normalize intensity to 0-255
            # normalized_data = ((flipped_data - flipped_data.min()) * (255 - 0)) / (flipped_data.max() - flipped_data.min()) + 0
            
            # Move the channel axis from 0 to 2
            flipped_data = np.transpose(data, (0, 2, 1))
            print(flipped_data.shape)
            
            # Create a new NIfTI image with the flipped data
            flipped_img = nib.Nifti1Image(flipped_data, img.affine, img.header)
            
            # Save the new image to a file
            # flipped_filepath = os.path.join(folder, 'flipped_' + filename)
            nib.save(flipped_img, filepath)
            print('Saved flipped image to', filepath)

# Replace 'your_folder' with the path to your folder
left_turn_180_and_fliplr(join(nnUNet_raw, 'Dataset501_LUNA/imagesTs_predfulres'))
left_turn_180_and_fliplr(join(nnUNet_raw, 'Dataset601_LUNA/imagesTs_predfulres'))

In [1]:
from nnunetv2.paths import nnUNet_results, nnUNet_raw
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
# 计算三维下各种指标
from __future__ import absolute_import, print_function
import pandas as pd
import GeodisTK
import numpy as np
from scipy import ndimage


# pixel accuracy
def binary_pa(s, g):
    """
        calculate the pixel accuracy of two N-d volumes.
        s: the segmentation volume of numpy array
        g: the ground truth volume of numpy array
        """
    pa = ((s == g).sum()) / g.size
    return pa


# Dice evaluation
def binary_dice(s, g):
    """
    calculate the Dice score of two N-d volumes.
    s: the segmentation volume of numpy array
    g: the ground truth volume of numpy array
    """
    assert (len(s.shape) == len(g.shape))
    prod = np.multiply(s, g)
    s0 = prod.sum()
    dice = (2.0 * s0 + 1e-10) / (s.sum() + g.sum() + 1e-10)
    return dice


# IOU evaluation
def binary_iou(s, g):
    assert (len(s.shape) == len(g.shape))
    # 两者相乘值为1的部分为交集
    intersecion = np.multiply(s, g)
    # 两者相加，值大于0的部分为交集
    union = np.asarray(s + g > 0, np.float32)
    iou = intersecion.sum() / (union.sum() + 1e-10)
    return iou


# Hausdorff and ASSD evaluation
def get_edge_points(img):
    """
    get edge points of a binary segmentation result
    """
    dim = len(img.shape)
    if (dim == 2):
        strt = ndimage.generate_binary_structure(2, 1)
    else:
        strt = ndimage.generate_binary_structure(3, 1)  # 三维结构元素，与中心点相距1个像素点的都是邻域
    ero = ndimage.binary_erosion(img, strt)
    edge = np.asarray(img, np.uint8) - np.asarray(ero, np.uint8)
    return edge


def binary_hausdorff95(s, g, spacing=None):
    """
    get the hausdorff distance between a binary segmentation and the ground truth
    inputs:
        s: a 3D or 2D binary image for segmentation
        g: a 2D or 2D binary image for ground truth
        spacing: a list for image spacing, length should be 3 or 2
    """
    s_edge = get_edge_points(s)
    g_edge = get_edge_points(g)
    image_dim = len(s.shape)
    assert (image_dim == len(g.shape))
    if (spacing == None):
        spacing = [1.0] * image_dim
    else:
        assert (image_dim == len(spacing))
    img = np.zeros_like(s)
    if (image_dim == 2):
        s_dis = GeodisTK.geodesic2d_raster_scan(img, s_edge, 0.0, 2)
        g_dis = GeodisTK.geodesic2d_raster_scan(img, g_edge, 0.0, 2)
    elif (image_dim == 3):
        s_dis = GeodisTK.geodesic3d_raster_scan(img, s_edge, spacing, 0.0, 2)
        g_dis = GeodisTK.geodesic3d_raster_scan(img, g_edge, spacing, 0.0, 2)

    dist_list1 = s_dis[g_edge > 0]
    dist_list1 = sorted(dist_list1)
    dist1 = dist_list1[int(len(dist_list1) * 0.95)]
    dist_list2 = g_dis[s_edge > 0]
    dist_list2 = sorted(dist_list2)
    dist2 = dist_list2[int(len(dist_list2) * 0.95)]
    return max(dist1, dist2)


# 平均表面距离
def binary_assd(s, g, spacing=None):
    """
    get the average symetric surface distance between a binary segmentation and the ground truth
    inputs:
        s: a 3D or 2D binary image for segmentation
        g: a 2D or 2D binary image for ground truth
        spacing: a list for image spacing, length should be 3 or 2
    """
    s_edge = get_edge_points(s)
    g_edge = get_edge_points(g)
    image_dim = len(s.shape)
    assert (image_dim == len(g.shape))
    if (spacing == None):
        spacing = [1.0] * image_dim
    else:
        assert (image_dim == len(spacing))
    img = np.zeros_like(s)
    if (image_dim == 2):
        s_dis = GeodisTK.geodesic2d_raster_scan(img, s_edge, 0.0, 2)
        g_dis = GeodisTK.geodesic2d_raster_scan(img, g_edge, 0.0, 2)
    elif (image_dim == 3):
        s_dis = GeodisTK.geodesic3d_raster_scan(img, s_edge, spacing, 0.0, 2)
        g_dis = GeodisTK.geodesic3d_raster_scan(img, g_edge, spacing, 0.0, 2)

    ns = s_edge.sum()
    ng = g_edge.sum()
    s_dis_g_edge = s_dis * g_edge
    g_dis_s_edge = g_dis * s_edge
    assd = (s_dis_g_edge.sum() + g_dis_s_edge.sum()) / (ns + ng)
    return assd


# relative volume error evaluation
def binary_relative_volume_error(s_volume, g_volume):
    s_v = float(s_volume.sum())
    g_v = float(g_volume.sum())
    assert (g_v > 0)
    rve = abs(s_v - g_v) / g_v
    return rve


def compute_class_sens_spec(pred, label):
    """
    Compute sensitivity and specificity for a particular example
    for a given class for binary.
    Args:
        pred (np.array): binary arrary of predictions, shape is
                         (height, width, depth).
        label (np.array): binary array of labels, shape is
                          (height, width, depth).
    Returns:
        sensitivity (float): precision for given class_num.
        specificity (float): recall for given class_num
    """
    tp = np.sum((pred == 1) & (label == 1))
    tn = np.sum((pred == 0) & (label == 0))
    fp = np.sum((pred == 1) & (label == 0))
    fn = np.sum((pred == 0) & (label == 1))
     # 添加错误检查以防止除以零
    if tp + fn != 0:
        sensitivity = tp / (tp + fn)
    else:
        sensitivity = 0

    if tn + fp != 0:
        specificity = tn / (tn + fp)
    else:
        specificity = 0
    # sensitivity = tp / (tp + fn)
    # specificity = tn / (tn + fp)

    return sensitivity, specificity


def get_evaluation_score(s_volume, g_volume, spacing, metric):
    s_volume = (s_volume > 0).astype(np.uint8)
    g_volume = (g_volume > 0).astype(np.uint8)
    if (len(s_volume.shape) == 4):
        assert (s_volume.shape[0] == 1 and g_volume.shape[0] == 1)
        s_volume = np.reshape(s_volume, s_volume.shape[1:])
        g_volume = np.reshape(g_volume, g_volume.shape[1:])
    if (s_volume.shape[0] == 1):
        s_volume = np.reshape(s_volume, s_volume.shape[1:])
        g_volume = np.reshape(g_volume, g_volume.shape[1:])
    metric_lower = metric.lower()

    if (metric_lower == "dice"):
        score = binary_dice(s_volume, g_volume)

    elif (metric_lower == "iou"):
        score = binary_iou(s_volume, g_volume)

    elif (metric_lower == 'assd'):
        score = binary_assd(s_volume, g_volume, spacing)

    elif (metric_lower == "hausdorff95"):
        score = binary_hausdorff95(s_volume, g_volume, spacing)

    elif (metric_lower == "rve"):
        score = binary_relative_volume_error(s_volume, g_volume)

    elif (metric_lower == "volume"):
        voxel_size = 1.0
        for dim in range(len(spacing)):
            voxel_size = voxel_size * spacing[dim]
        score = g_volume.sum() * voxel_size
    else:
        raise ValueError("unsupported evaluation metric: {0:}".format(metric))

    return score



# import os
# import nibabel as nib
# # join(nnUNet_raw, 'Dataset501_LUNA/imagesTs'),
# #             join(nnUNet_raw, 'Dataset501_LUNA/imagesTs_predfulres'),
# seg_path = join(nnUNet_raw, 'Dataset601_LUNA/imagesTs_predfulres')
# gd_path = join(nnUNet_raw, 'Dataset601_LUNA/groundtruthseg')
# save_dir = './'
# seg = sorted(os.listdir(seg_path))

# dices = []
# hds = []
# rves = []
# case_name = []
# senss = []
# specs = []
# for name in seg:
#     if not name.startswith('.') and name.endswith('nii.gz'):
#         # 加载label and segmentation image
#         seg_ = nib.load(os.path.join(seg_path, name))
#         seg_arr = seg_.get_fdata().astype('float32')
#         gd_ = nib.load(os.path.join(gd_path, name))
#         gd_arr = gd_.get_fdata().astype('float32')
#         case_name.append(name)

#         # 求hausdorff95距离
#         hd_score = get_evaluation_score(seg_arr, gd_arr, spacing=None, metric='hausdorff95')
#         hds.append(hd_score)

#         # 求体积相关误差
#         rve = get_evaluation_score(seg_arr, gd_arr, spacing=None, metric='rve')
#         rves.append(rve)

#         # 求dice
#         dice = get_evaluation_score(seg_.get_fdata(), gd_.get_fdata(), spacing=None, metric='dice')
#         dices.append(dice)

#         # 敏感度，特异性
#         sens, spec = compute_class_sens_spec(seg_.get_fdata(), gd_.get_fdata())
#         senss.append(sens)
#         specs.append(spec)
# # 存入pandas
# data = {'dice': dices, 'RVE': rves, 'Sens': senss, 'Spec': specs, 'HD95': hds}
# df = pd.DataFrame(data=data, columns=['dice', 'RVE', 'Sens', 'Spec', 'HD95'], index=case_name)
# df.to_csv(os.path.join(save_dir, 'metrics1.csv'))


In [2]:
# import os
# import nibabel as nib
# import pandas as pd

# def calculate_metrics(seg_path, gd_path, save_dir):
#     seg = sorted(os.listdir(seg_path))

#     dices = []
#     hds = []
#     rves = []
#     case_name = []
#     senss = []
#     specs = []
#     for name in seg:
#         if not name.startswith('.') and name.endswith('nii.gz'):
#             # 加载label and segmentation image
#             seg_ = nib.load(os.path.join(seg_path, name))
#             seg_arr = seg_.get_fdata().astype('float32')
#             gd_ = nib.load(os.path.join(gd_path, name))
#             gd_arr = gd_.get_fdata().astype('float32')
#             case_name.append(name)

#             # 求hausdorff95距离
#             hd_score = get_evaluation_score(seg_arr, gd_arr, spacing=None, metric='hausdorff95')
#             hds.append(hd_score)

#             # 求体积相关误差
#             rve = get_evaluation_score(seg_arr, gd_arr, spacing=None, metric='rve')
#             rves.append(rve)

#             # 求dice
#             dice = get_evaluation_score(seg_.get_fdata(), gd_.get_fdata(), spacing=None, metric='dice')
#             dices.append(dice)

#             # 敏感度，特异性
#             sens, spec = compute_class_sens_spec(seg_.get_fdata(), gd_.get_fdata())
#             senss.append(sens)
#             specs.append(spec)
#     # 存入pandas
#     data = {'dice': dices, 'RVE': rves, 'Sens': senss, 'Spec': specs, 'HD95': hds}
#     df = pd.DataFrame(data=data, columns=['dice', 'RVE', 'Sens', 'Spec', 'HD95'], index=case_name)
#     df.to_csv(os.path.join(save_dir, 'metrics_911.csv'))
# # 使用函数
# seg_path = '/home/xulei/projects/wei/2DSAM3D/output_gt/'
# gd_path = join(nnUNet_raw, 'Dataset501_LUNA/labelTs')
# # seg_path = join(nnUNet_raw, 'Dataset501_LUNA/Ts_predfulres')
# # gd_path = join(nnUNet_raw, 'Dataset501_LUNA/labelTs')
# save_dir = './output'
# calculate_metrics(seg_path, gd_path, save_dir)
# # seg_path_1 = join(nnUNet_raw, 'Dataset601_LUNA/Ts_predfulresv')
# seg_path_1 = '/home/xulei/projects/wei/2DSAM3D/output_sam/'
# gd_path_1 = join(nnUNet_raw, 'Dataset601_LUNA/labelTs')
# save_dir = './output1'
# calculate_metrics(seg_path, gd_path, save_dir)


In [3]:
import os
import nibabel as nib
import pandas as pd
from tqdm import tqdm  # 导入 tqdm

def calculate_metrics(seg_path, gd_path, save_dir):
    seg = sorted(os.listdir(seg_path))

    dices = []
    hds = []
    rves = []
    case_name = []
    senss = []
    specs = []
    
    # 使用 tqdm() 包装你的迭代器，显示进度条
    for name in tqdm(seg, desc="Calculating Metrics", unit="image"):
        if not name.startswith('.') and name.endswith('nii.gz'):
            # 加载label and segmentation image
            seg_ = nib.load(os.path.join(seg_path, name))
            seg_arr = seg_.get_fdata().astype('uint8')
            gd_ = nib.load(os.path.join(gd_path, name))
            gd_arr = gd_.get_fdata().astype('uint8')
            case_name.append(name)

            # 求hausdorff95距离
            hd_score = get_evaluation_score(seg_arr, gd_arr, spacing=None, metric='hausdorff95')
            hds.append(hd_score)

            # 求体积相关误差
            rve = get_evaluation_score(seg_arr, gd_arr, spacing=None, metric='rve')
            rves.append(rve)

            # 求dice
            dice = get_evaluation_score(seg_.get_fdata(), gd_.get_fdata(), spacing=None, metric='dice')
            dices.append(dice)

            # 敏感度，特异性
            sens, spec = compute_class_sens_spec(seg_.get_fdata(), gd_.get_fdata())
            senss.append(sens)
            specs.append(spec)
    
    # 存入pandas
    data = {'dice': dices, 'RVE': rves, 'Sens': senss, 'Spec': specs, 'HD95': hds}
    df = pd.DataFrame(data=data, columns=['dice', 'RVE', 'Sens', 'Spec', 'HD95'], index=case_name)
    df.to_csv(os.path.join(save_dir, 'metrics_911.csv'))

# 使用函数
seg_path = '/home/xulei/projects/wei/2DSAM3D/output_gt/'
gd_path = join(nnUNet_raw, 'Dataset501_LUNA/labelTs')
save_dir = './output'
calculate_metrics(seg_path, gd_path, save_dir)

seg_path_1 = '/home/xulei/projects/wei/2DSAM3D/output_sam/'
gd_path_1 = join(nnUNet_raw, 'Dataset601_LUNA/labelTs')
save_dir_1 = './output1'
calculate_metrics(seg_path_1, gd_path_1, save_dir_1)


Calculating Metrics:   0%|          | 0/35 [00:00<?, ?image/s]

Calculating Metrics:  80%|████████  | 28/35 [33:49<09:19, 79.91s/image] 